# TD1 — La Modélisation Dimensionnelle Avancée (Schéma en Étoile)

**Durée estimée** : 1h30  
**Niveau** : Intermédiaire  
**Objectifs pédagogiques** :
1.  **Concevoir** un schéma en étoile réaliste avec plusieurs niveaux de complexité.
2.  **Implémenter** des tables de dimensions et de faits avec des concepts avancés.
3.  **Maîtriser** les concepts fondamentaux :
    *   **Grain** : Définition précise et gestion des multi-grains.
    *   **Dimensions** : Hiérarchies, attributs dégénérés, dimensions conformes.
    *   **Faits** : Mesures additives, semi-additives, non additives.
    *   **Clés de Substitution** : Gestion des SCD (Slowly Changing Dimensions).
    *   **Performance** : Indexation et partitionnement stratégique.

---

In [ ]:
# Import SQLite (exécution minimale)
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
print('Base TD1 créée (SQLite en mémoire)')

Base TD1 créée


## Partie 1 : Le Modèle Conceptuel Avancé (Mermaid)

Nous allons modéliser un processus de vente retail avec des caractéristiques réelles :
- Ventes multi-canaux (magasin physique, e-commerce, mobile)
- Gestion des promotions et remises
- Suivi des stocks et des retours clients
- Analyse géographique à plusieurs niveaux

### Les choix de modélisation avancés :
- **Fait** : `FACT_VENTES` avec mesures additives (`montant_ht`, `quantite`, `montant_remise`) et semi-additives (`stock_eod`).
- **Dimensions** :
  - `DIM_DATE` : Hiérarchie temporelle complète avec attributs calendaires et fiscaux.
  - `DIM_PRODUIT` : Hiérarchie produit, gestion des SCD Type 2, attributs de performance.
  - `DIM_MAGASIN` : Géolocalisation, caractéristiques, surface, type.
  - `DIM_CLIENT` : Démographie, segment, RFM.
  - `DIM_PROMOTION` : Types de promotions, règles de calcul.
  - `DIM_CANAL` : Canal de vente avec caractéristiques techniques.

### Attributs dégénérés :
- `ticket_id` : Identifiant du ticket de caisse (stocké dans la table de faits).
- `transaction_time` : Heure de la transaction (sans dimension temps dédiée).

### Le Diagramme Entité-Relation Avancé :
```mermaid
erDiagram
  DIM_DATE ||--o{ FACT_VENTES : date_id
  DIM_PRODUIT ||--o{ FACT_VENTES : produit_sk
  DIM_MAGASIN ||--o{ FACT_VENTES : magasin_sk
  DIM_CLIENT ||--o{ FACT_VENTES : client_sk
  DIM_PROMOTION ||--o{ FACT_VENTES : promotion_sk
  DIM_CANAL ||--o{ FACT_VENTES : canal_sk

  DIM_DATE {
    int date_id PK "Clé technique"
    date date_cal "Date réelle"
    int annee
    int trimestre
    int mois
    int jour_semaine
    int jour_mois
    string nom_jour
    boolean est_jour_ferie
    boolean est_weekend
    int semaine_iso
    string periode_fiscale
  }
  DIM_PRODUIT {
    int produit_sk PK "Clé de substitution"
    string produit_id "Code produit (Source)"
    string produit_nom
    string categorie
    string sous_categorie
    string marque
    decimal prix_unitaire_ht
    decimal poids_kg
    string couleur
    string taille
    boolean est_perissable
    date debut_validite
    date fin_validite
    int version_sc "SCD Type 2"
  }
  DIM_MAGASIN {
    int magasin_sk PK "Clé de substitution"
    string magasin_id "Code magasin (Source)"
    string nom_magasin
    string adresse
    string ville
    string departement
    string region
    string pays
    decimal surface_m2
    string type_magasin
    date ouverture_date
    boolean est_actif
  }
  DIM_CLIENT {
    int client_sk PK "Clé de substitution"
    string client_id "Code client (Source)"
    string nom
    string prenom
    date date_naissance
    string segment_client
    string ville
    string region
    date premiere_achat
    int score_rfm
    boolean est_actif
  }
  DIM_PROMOTION {
    int promotion_sk PK "Clé de substitution"
    string promotion_id "Code promotion"
    string nom_promotion
    string type_remise
    decimal taux_remise
    decimal montant_remise
    date debut_promo
    date fin_promo
    boolean est_cumulable
  }
  DIM_CANAL {
    int canal_sk PK "Clé de substitution"
    string canal_id "Code canal"
    string nom_canal
    string type_canal
    boolean est_phisique
    boolean est_en_ligne
  }
  FACT_VENTES {
    int fact_sk PK
    int date_id FK
    int produit_sk FK
    int magasin_sk FK
    int client_sk FK
    int promotion_sk FK
    int canal_sk FK
    string ticket_id "Attribut dégénéré"
    time transaction_time "Attribut dégénéré"
    int quantite "Mesure additive"
    decimal montant_ht "Mesure additive"
    decimal montant_tva "Mesure additive"
    decimal montant_remise "Mesure additive"
    decimal montant_total "Mesure additive"
    decimal stock_eod "Mesure semi-additive"
    int nb_retours "Mesure additive"
  }
```

<!-- Cellule de transition -->
Dans cette partie, nous allons créer les tables.
Nous utilisons des `INTEGER PRIMARY KEY AUTOINCREMENT` pour gérer automatiquement les clés de substitution (Surrogate Keys).

### Concepts Clés Avancés :

1.  **Le Grain (Granularité) Précis** :
    -   Définition : **Une ligne = Une ligne de ticket de caisse pour un produit spécifique, vendu via un canal spécifique, à un client spécifique, dans un magasin spécifique, à une date et heure données.**
    -   Impact : Permet l'analyse au niveau le plus fin tout en permettant les agrégations flexibles.
    -   Attention : Ne pas mélanger les grains (ex: ticket complet vs ligne produit).

2.  **Clés de Substitution et SCD** :
    -   `produit_sk` vs `produit_id` : Indépendance, performance, historisation.
    -   **SCD Type 2** : `version_sc` dans `DIM_PRODUIT` pour tracer les changements de prix/catégorie.
    -   **SCD Type 1** : Mise à jour directe pour les corrections d'erreurs.

3.  **Types de Mesures** :
    -   **Additives** : `quantite`, `montant_ht`, `montant_tva` - peuvent être sommées sur toutes les dimensions.
    -   **Semi-additives** : `stock_eod` - sommables sur certaines dimensions seulement (produit, magasin) mais pas sur le temps.
    -   **Non additives** : `prix_unitaire_ht` - ne doivent jamais être stockées dans les faits.

4.  **Attributs Dégénérés** :
    -   `ticket_id` : Pas de dimension dédiée, stocké directement dans les faits.
    -   `transaction_time` : Heure sans dimension temps dédiée (granularité journalière).

5.  **Dimensions Conformes** :
    -   `DIM_DATE`, `DIM_PRODUIT`, `DIM_MAGASIN` peuvent être partagées entre plusieurs faits.
    -   Permettent l'analyse cross-facts (ex: ventes vs stocks).

In [ ]:
# Partie 2 : Implémentation SQL Avancée (DDL)

# Dimension Temps avec attributs calendaires et fiscaux
cursor.execute('''
CREATE TABLE dim_date (
  date_id INTEGER PRIMARY KEY AUTOINCREMENT,
  date_cal DATE NOT NULL UNIQUE,
  annee INTEGER NOT NULL,
  trimestre INTEGER NOT NULL,
  mois INTEGER NOT NULL,
  jour_semaine INTEGER NOT NULL, -- 1=Monday, 7=Sunday
  jour_mois INTEGER NOT NULL,
  nom_jour TEXT NOT NULL,
  est_jour_ferie BOOLEAN DEFAULT 0,
  est_weekend BOOLEAN DEFAULT 0,
  semaine_iso INTEGER NOT NULL,
  periode_fiscale TEXT, -- Ex: "2024-Q1", "2024-M1"
  CONSTRAINT chk_date CHECK (date_cal IS NOT NULL)
);
''')

# Dimension Produit avec SCD Type 2
cursor.execute('''
CREATE TABLE dim_produit (
  produit_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  produit_id TEXT NOT NULL, -- Business Key
  produit_nom TEXT NOT NULL,
  categorie TEXT NOT NULL,
  sous_categorie TEXT,
  marque TEXT,
  prix_unitaire_ht DECIMAL(10,2),
  poids_kg DECIMAL(8,3),
  couleur TEXT,
  taille TEXT,
  est_perissable BOOLEAN DEFAULT 0,
  debut_validite DATE NOT NULL,
  fin_validite DATE, -- NULL si actif
  version_sc INTEGER DEFAULT 1, -- SCD Type 2
  CONSTRAINT chk_produit CHECK (prix_unitaire_ht >= 0),
  CONSTRAINT chk_poids CHECK (poids_kg >= 0)
);
''')

# Dimension Magasin avec géolocalisation
cursor.execute('''
CREATE TABLE dim_magasin (
  magasin_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  magasin_id TEXT NOT NULL UNIQUE,
  nom_magasin TEXT NOT NULL,
  adresse TEXT,
  ville TEXT NOT NULL,
  departement TEXT,
  region TEXT NOT NULL,
  pays TEXT DEFAULT 'France',
  surface_m2 DECIMAL(8,2),
  type_magasin TEXT, -- 'Flagship', 'Outlet', 'Drive'
  ouverture_date DATE NOT NULL,
  est_actif BOOLEAN DEFAULT 1,
  CONSTRAINT chk_surface CHECK (surface_m2 > 0)
);
''')

# Dimension Client avec segmentation RFM
cursor.execute('''
CREATE TABLE dim_client (
  client_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  client_id TEXT NOT NULL UNIQUE,
  nom TEXT NOT NULL,
  prenom TEXT NOT NULL,
  date_naissance DATE,
  segment_client TEXT, -- 'Premium', 'Standard', 'Occasional'
  ville TEXT,
  region TEXT,
  premiere_achat DATE,
  score_rfm INTEGER, -- 1-5 score
  est_actif BOOLEAN DEFAULT 1,
  CONSTRAINT chk_rfm CHECK (score_rfm BETWEEN 1 AND 5)
);
''')

# Dimension Promotion
cursor.execute('''
CREATE TABLE dim_promotion (
  promotion_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  promotion_id TEXT NOT NULL UNIQUE,
  nom_promotion TEXT NOT NULL,
  type_remise TEXT NOT NULL, -- 'Pourcentage', 'MontantFixe', '2Pour1'
  taux_remise DECIMAL(5,4), -- 0.0000 to 1.0000
  montant_remise DECIMAL(10,2),
  debut_promo DATE NOT NULL,
  fin_promo DATE NOT NULL,
  est_cumulable BOOLEAN DEFAULT 0,
  CONSTRAINT chk_remise CHECK (
    (type_remise = 'Pourcentage' AND taux_remise BETWEEN 0 AND 1) OR
    (type_remise = 'MontantFixe' AND montant_remise > 0) OR
    (type_remise = '2Pour1')
  )
);
''')

# Dimension Canal
cursor.execute('''
CREATE TABLE dim_canal (
  canal_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  canal_id TEXT NOT NULL UNIQUE,
  nom_canal TEXT NOT NULL,
  type_canal TEXT NOT NULL, -- 'Physique', 'EnLigne', 'Mobile'
  est_physique BOOLEAN DEFAULT 0,
  est_en_ligne BOOLEAN DEFAULT 0
);
''')

# Table de Faits avec mesures multiples et attributs dégénérés
cursor.execute('''
CREATE TABLE fact_ventes (
  fact_sk INTEGER PRIMARY KEY AUTOINCREMENT,
  date_id INTEGER NOT NULL,
  produit_sk INTEGER NOT NULL,
  magasin_sk INTEGER NOT NULL,
  client_sk INTEGER NOT NULL,
  promotion_sk INTEGER, -- Nullable si pas de promotion
  canal_sk INTEGER NOT NULL,
  ticket_id TEXT NOT NULL, -- Attribut dégénéré
  transaction_time TIME NOT NULL, -- Attribut dégénéré
  
  -- Mesures additives
  quantite INTEGER NOT NULL,
  montant_ht DECIMAL(12,2) NOT NULL,
  montant_tva DECIMAL(12,2) NOT NULL,
  montant_remise DECIMAL(12,2) DEFAULT 0,
  montant_total DECIMAL(12,2) NOT NULL,
  nb_retours INTEGER DEFAULT 0,
  
  -- Mesure semi-additive
  stock_eod DECIMAL(12,2) DEFAULT 0, -- Stock fin de journée
  
  -- Contraintes d'intégrité référentielle
  FOREIGN KEY (date_id) REFERENCES dim_date(date_id),
  FOREIGN KEY (produit_sk) REFERENCES dim_produit(produit_sk),
  FOREIGN KEY (magasin_sk) REFERENCES dim_magasin(magasin_sk),
  FOREIGN KEY (client_sk) REFERENCES dim_client(client_sk),
  FOREIGN KEY (promotion_sk) REFERENCES dim_promotion(promotion_sk),
  FOREIGN KEY (canal_sk) REFERENCES dim_canal(canal_sk),
  
  -- Contraintes de validation
  CONSTRAINT chk_quantite CHECK (quantite > 0),
  CONSTRAINT chk_montants CHECK (montant_ht >= 0 AND montant_tva >= 0 AND montant_total >= 0),
  CONSTRAINT chk_calcul_total CHECK (montant_total = montant_ht + montant_tva - montant_remise),
  CONSTRAINT chk_stock CHECK (stock_eod >= 0)
);
''')

print('✅ Tables avancées créées avec succès.')

Tables créées


<!-- Cellule de transition -->
Une fois la structure en place, nous devons l'alimenter.
Dans un projet réel, cette étape est réalisée par l'outil ETL (Extract-Transform-Load).
Ici, nous simulons le chargement avec des instructions `INSERT`.

In [ ]:
# Partie 3 : Chargement Avancé des Données (ETL Realiste)

# 1. Chargement de la Dimension Temps avec calculs calendaires
dates_data = [
    ('2024-01-02', 2024, 1, 1, 2, 'Mardi', 0, 0, 1, '2024-Q1'),
    ('2024-01-03', 2024, 1, 1, 3, 'Mercredi', 0, 0, 1, '2024-Q1'),
    ('2024-01-04', 2024, 1, 1, 4, 'Jeudi', 0, 0, 1, '2024-Q1'),
    ('2024-01-05', 2024, 1, 1, 5, 'Vendredi', 0, 0, 1, '2024-Q1'),
    ('2024-01-06', 2024, 1, 1, 6, 'Samedi', 0, 1, 1, '2024-Q1'),
    ('2024-01-07', 2024, 1, 1, 7, 'Dimanche', 0, 1, 1, '2024-Q1'),
]

cursor.executemany('''
INSERT INTO dim_date (date_cal, annee, trimestre, mois, jour_semaine, jour_mois, 
                     nom_jour, est_jour_ferie, est_weekend, semaine_iso, periode_fiscale) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
''', dates_data)

# 2. Chargement de la Dimension Produit avec SCD Type 2
produits_data = [
    ('P001', 'Chemise Oxford Premium', 'Textile', 'Chemises', 'Oxford', 89.99, 0.250, 'Blanc', 'L', 0, '2024-01-01', None, 1),
    ('P002', 'Sneakers Run Pro', 'Chaussure', 'Sneakers', 'SportMax', 129.99, 0.450, 'Noir', '42', 0, '2024-01-01', None, 1),
    ('P003', 'Jean Slim Fit', 'Textile', 'Jeans', 'DenimCo', 79.99, 0.680, 'Bleu', '32', 0, '2024-01-01', None, 1),
    ('P004', 'T-shirt Basic', 'Textile', 'T-shirts', 'EcoWear', 19.99, 0.150, 'Blanc', 'M', 0, '2024-01-01', None, 1),
    ('P005', 'Veste Cuir', 'Textile', 'Vestes', 'LeatherPro', 299.99, 1.200, 'Noir', 'L', 0, '2024-01-01', None, 1),
]

cursor.executemany('''
INSERT INTO dim_produit (produit_id, produit_nom, categorie, sous_categorie, marque, 
                        prix_unitaire_ht, poids_kg, couleur, taille, est_perissable, 
                        debut_validite, fin_validite, version_sc) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
''', produits_data)

# 3. Chargement de la Dimension Magasin
magasins_data = [
    ('M001', 'Paris Champs-Élysées', '75 Av. des Champs-Élysées', 'Paris', '75', 'Île-de-France', 'France', 1500.0, 'Flagship', '2020-03-15', 1),
    ('M002', 'Lyon Confluence', '15 Quai Perrache', 'Lyon', '69', 'Auvergne-Rhône-Alpes', 'France', 800.0, 'Standard', '2021-06-20', 1),
    ('M003', 'Marseille Vieux-Port', '2 Quai du Port', 'Marseille', '13', 'Provence-Alpes-Côte d\'Azur', 'France', 600.0, 'Standard', '2019-11-10', 1),
    ('M004', 'Boutique Online', 'Warehouse Central', 'Paris', '75', 'Île-de-France', 'France', 2000.0, 'E-commerce', '2020-01-01', 1),
]

cursor.executemany('''
INSERT INTO dim_magasin (magasin_id, nom_magasin, adresse, ville, departement, region, 
                        pays, surface_m2, type_magasin, ouverture_date, est_actif) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
''', magasins_data)

# 4. Chargement de la Dimension Client avec segmentation
clients_data = [
    ('C001', 'Martin', 'Sophie', '1985-07-15', 'Premium', 'Paris', 'Île-de-France', '2023-01-15', 5, 1),
    ('C002', 'Dubois', 'Pierre', '1990-03-22', 'Standard', 'Lyon', 'Auvergne-Rhône-Alpes', '2023-02-10', 3, 1),
    ('C003', 'Bernard', 'Marie', '1988-11-30', 'Premium', 'Marseille', 'Provence-Alpes-Côte d\'Azur', '2023-01-20', 4, 1),
    ('C004', 'Petit', 'Jean', '1992-05-08', 'Occasional', 'Paris', 'Île-de-France', '2023-03-05', 2, 1),
    ('C005', 'Robert', 'Claire', '1987-09-12', 'Standard', 'Lyon', 'Auvergne-Rhône-Alpes', '2023-02-28', 3, 1),
]

cursor.executemany('''
INSERT INTO dim_client (client_id, nom, prenom, date_naissance, segment_client, ville, 
                      region, premiere_achat, score_rfm, est_actif) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
''', clients_data)

# 5. Chargement de la Dimension Promotion
promotions_data = [
    ('PROMO001', 'Soldes Hiver -20%', 'Pourcentage', 0.20, None, '2024-01-02', '2024-01-07', 1),
    ('PROMO002', 'Fidélité -10€', 'MontantFixe', None, 10.00, '2024-01-01', '2024-01-31', 0),
    ('PROMO003', '2 pour 1 T-shirts', '2Pour1', None, None, '2024-01-03', '2024-01-05', 1),
]

cursor.executemany('''
INSERT INTO dim_promotion (promotion_id, nom_promotion, type_remise, taux_remise, montant_remise, 
                          debut_promo, fin_promo, est_cumulable) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?);
''', promotions_data)

# 6. Chargement de la Dimension Canal
canaux_data = [
    ('CAN001', 'Magasin Physique', 'Physique', 1, 0),
    ('CAN002', 'Site Web', 'EnLigne', 0, 1),
    ('CAN003', 'Application Mobile', 'Mobile', 0, 1),
]

cursor.executemany('''
INSERT INTO dim_canal (canal_id, nom_canal, type_canal, est_physique, est_en_ligne) 
VALUES (?, ?, ?, ?, ?);
''', canaux_data)

# 7. Chargement de la Table de Faits avec calculs complexes
ventes_data = [
    # Ticket T001 - 2024-01-02 - Client Premium avec promotion
    (1, 1, 1, 1, 1, 1, 'T001', '14:30:00', 1, 89.99, 17.99, 18.00, 89.98, 120.0, 0),
    (1, 4, 1, 1, 1, 1, 'T001', '14:31:00', 2, 39.98, 7.99, 8.00, 39.97, 180.0, 0),
    
    # Ticket T002 - 2024-01-03 - Client Standard sans promotion
    (2, 2, 2, 2, None, 1, 'T002', '10:15:00', 1, 129.99, 26.00, 0.00, 155.99, 85.0, 0),
    
    # Ticket T003 - 2024-01-04 - Client Premium avec promotion 2pour1
    (3, 3, 3, 3, 3, 1, 'T003', '16:45:00', 1, 79.99, 16.00, 39.99, 56.00, 95.0, 0),
    
    # Ticket T004 - 2024-01-05 - Client Occasional
    (4, 5, 1, 4, None, 1, 'T004', '12:20:00', 1, 299.99, 60.00, 0.00, 359.99, 50.0, 1),
    
    # Ticket T005 - 2024-01-06 - Vente en ligne
    (5, 2, 4, 5, 2, 2, 'T005', '18:30:00', 1, 129.99, 26.00, 13.00, 142.99, 200.0, 0),
    
    # Ticket T006 - 2024-01-07 - Client fidèle avec remise cumulée
    (6, 1, 2, 5, 2, 3, 'T006', '11:10:00', 1, 89.99, 17.99, 17.99, 89.99, 75.0, 0),
]

cursor.executemany('''
INSERT INTO fact_ventes (date_id, produit_sk, magasin_sk, client_sk, promotion_sk, canal_sk, 
                        ticket_id, transaction_time, quantite, montant_ht, montant_tva, 
                        montant_remise, montant_total, stock_eod, nb_retours) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
''', ventes_data)

print('✅ Données avancées chargées avec succès.')
print(f'   - {len(dates_data)} dates calendaires')
print(f'   - {len(produits_data)} produits (avec SCD Type 2)')
print(f'   - {len(magasins_data)} magasins (géolocalisés)')
print(f'   - {len(clients_data)} clients (segmentés RFM)')
print(f'   - {len(promotions_data)} promotions')
print(f'   - {len(canaux_data)} canaux de vente')
print(f'   - {len(ventes_data)} transactions ventes')

Données insérées


<!-- Cellule de transition -->
Le modèle est chargé. Il faut maintenant vérifier qu'il répond aux questions métier.
Nous allons exécuter 3 requêtes types d'analyse décisionnelle.

In [ ]:
# Partie 4 : Validation Avancée du Modèle

# 1. Contrôles de base et volumétrie
cursor.execute('SELECT COUNT(*) AS nb_ventes FROM fact_ventes;')
nb_ventes = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(DISTINCT ticket_id) AS nb_tickets FROM fact_ventes;')
nb_tickets = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(*) AS nb_produits FROM dim_produit WHERE fin_validite IS NULL;')
nb_produits_actifs = cursor.fetchone()[0]

print(f'📊 Volumétrie du data warehouse :')
print(f'   - {nb_ventes} lignes de ventes')
print(f'   - {nb_tickets} tickets uniques')
print(f'   - {nb_produits_actifs} produits actifs')

# 2. Analyse des ventes par segment client (avec jointures multiples)
print('\n👥 Chiffre d\'Affaires par Segment Client :')
cursor.execute('''
SELECT 
    c.segment_client,
    COUNT(DISTINCT f.ticket_id) AS nb_tickets,
    SUM(f.quantite) AS total_quantite,
    SUM(f.montant_total) AS ca_total,
    AVG(f.montant_total) AS panier_moyen
FROM fact_ventes f
JOIN dim_client c ON f.client_sk = c.client_sk
GROUP BY c.segment_client
ORDER BY ca_total DESC;
''')
for row in cursor.fetchall():
    print(f"   {row[0]:12} | {row[1]:3} tickets | {row[2]:3} pcs | {row[3]:8.2f}€ | Panier: {row[4]:6.2f}€")

# 3. Performance par canal (incluant dimensions conformes)
print('\n🛒 Performance par Canal de Vente :')
cursor.execute('''
SELECT 
    can.nom_canal,
    COUNT(DISTINCT f.ticket_id) AS nb_transactions,
    SUM(f.montant_total) AS ca_total,
    SUM(f.montant_remise) AS remise_totale,
    CASE 
        WHEN SUM(f.montant_total) > 0 THEN 
            ROUND(SUM(f.montant_remise) / SUM(f.montant_total) * 100, 2)
        ELSE 0 
    END AS taux_remise_pct
FROM fact_ventes f
JOIN dim_canal can ON f.canal_sk = can.canal_sk
GROUP BY can.nom_canal
ORDER BY ca_total DESC;
''')
for row in cursor.fetchall():
    print(f"   {row[0]:20} | {row[1]:3} tx | {row[2]:8.2f}€ | {row[3]:6.2f}€ | {row[4]:5.1f}%")

# 4. Analyse temporelle avancée (jours fériés vs weekend)
print('\n📅 Analyse Temporelle :')
cursor.execute('''
SELECT 
    d.nom_jour,
    d.est_weekend,
    d.est_jour_ferie,
    COUNT(DISTINCT f.ticket_id) AS nb_transactions,
    SUM(f.montant_total) AS ca_total,
    AVG(f.montant_total) AS panier_moyen
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
GROUP BY d.nom_jour, d.est_weekend, d.est_jour_ferie
ORDER BY d.date_id;
''')
for row in cursor.fetchall():
    type_jour = "Weekend" if row[1] else ("Férié" if row[2] else "Semaine")
    print(f"   {row[0]:10} ({type_jour:8}) | {row[3]:3} tx | {row[4]:8.2f}€ | {row[5]:6.2f}€")

# 5. Analyse des promotions (mesure d'impact)
print('\n? Impact des Promotions :')
cursor.execute('''
SELECT 
    COALESCE(p.nom_promotion, 'Sans promotion') AS promotion,
    COUNT(DISTINCT f.ticket_id) AS nb_tickets,
    SUM(f.quantite) AS quantite_vendue,
    SUM(f.montant_total) AS ca_total,
    AVG(f.montant_total) AS panier_moyen,
    SUM(f.montant_remise) AS remise_totale
FROM fact_ventes f
LEFT JOIN dim_promotion p ON f.promotion_sk = p.promotion_sk
GROUP BY p.nom_promotion
ORDER BY ca_total DESC;
''')
for row in cursor.fetchall():
    print(f"   {row[0]:25} | {row[1]:3} tx | {row[2]:3} pcs | {row[3]:8.2f}€ | {row[4]:6.2f}€ | {row[5]:6.2f}€")

# 6. Contrôle de qualité : Mesures semi-additives (stock)
print('\n📦 Contrôle des Stocks (Mesure Semi-Additive) :')
cursor.execute('''
SELECT 
    p.produit_nom,
    m.nom_magasin,
    d.date_cal,
    f.stock_eod
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
JOIN dim_date d ON f.date_id = d.date_id
WHERE f.stock_eod > 0
ORDER BY d.date_cal, m.nom_magasin, p.produit_nom;
''')
for row in cursor.fetchall():
    print(f"   {row[2]} | {row[1]:20} | {row[0]:20} | Stock: {row[3]:6.0f}")

# 7. Validation des contraintes métier
print('\n✅ Validation des Contraintes Métier :')

# Vérifier que le montant total = HT + TVA - Remise
cursor.execute('''
SELECT COUNT(*) AS nb_erreurs
FROM fact_ventes 
WHERE ROUND(montant_total, 2) != ROUND(montant_ht + montant_tva - montant_remise, 2);
''')
nb_erreurs_calcul = cursor.fetchone()[0]

# Vérifier qu'il n'y a pas de ventes avec quantité négative
cursor.execute('SELECT COUNT(*) FROM fact_ventes WHERE quantite <= 0;')
nb_erreurs_quantite = cursor.fetchone()[0]

# Vérifier l'intégrité référentielle
cursor.execute('''
SELECT COUNT(*) AS nb_orphelins
FROM fact_ventes f
LEFT JOIN dim_produit p ON f.produit_sk = p.produit_sk
WHERE p.produit_sk IS NULL;
''')
nb_orphelins_produit = cursor.fetchone()[0]

print(f"   - Erreurs de calcul montant : {nb_erreurs_calcul}")
print(f"   - Erreurs de quantité : {nb_erreurs_quantite}")
print(f"   - Orphelins (produit) : {nb_orphelins_produit}")

conn.close()
print('\n🎉 TD1 Avancé terminé avec succès !')
print('\n📚 Concepts maîtrisés :')
print('   ✅ Schéma en étoile complexe (6 dimensions)')
print('   ✅ SCD Type 2 (historisation produits)')
print('   ✅ Attributs dégénérés (ticket_id, transaction_time)')
print('   ✅ Mesures additives et semi-additives')
print('   ✅ Contraintes et validation qualité')
print('   ✅ Jointures multiples et agrégations avancées')

Volume total : 4
Top 3 produits :
('Chemise Oxford', 180.0)
('Jeans Slim', 90.0)
('Sneakers Run', 75.0)
CA par ville/mois :
(2024, 1, 'Lyon', 60.0)
(2024, 1, 'Paris', 285.0)
TD1 terminé


## Partie 5 : Optimisation et Indexation Stratégique

### Index recommandés pour la performance :

```sql
-- Index sur les clés étrangères de la table de faits
CREATE INDEX idx_fact_ventes_date_id ON fact_ventes(date_id);
CREATE INDEX idx_fact_ventes_produit_sk ON fact_ventes(produit_sk);
CREATE INDEX idx_fact_ventes_magasin_sk ON fact_ventes(magasin_sk);
CREATE INDEX idx_fact_ventes_client_sk ON fact_ventes(client_sk);
CREATE INDEX idx_fact_ventes_canal_sk ON fact_ventes(canal_sk);

-- Index composite pour les requêtes fréquentes
CREATE INDEX idx_fact_ventes_date_produit ON fact_ventes(date_id, produit_sk);
CREATE INDEX idx_fact_ventes_magasin_date ON fact_ventes(magasin_sk, date_id);

-- Index sur les attributs de dimension fréquemment filtrés
CREATE INDEX idx_dim_client_segment ON dim_client(segment_client);
CREATE INDEX idx_dim_produit_categorie ON dim_produit(categorie);
CREATE INDEX idx_dim_magasin_region ON dim_magasin(region);
CREATE INDEX idx_dim_date_annee_mois ON dim_date(annee, mois);

-- Index sur les clés business (pour les lookups ETL)
CREATE INDEX idx_dim_produit_business_key ON dim_produit(produit_id);
CREATE INDEX idx_dim_client_business_key ON dim_client(client_id);
```

### Partitionnement suggéré pour les grandes volumes :

```sql
-- Partitionnement mensuel de la table de faits
CREATE TABLE fact_ventes_y2024m01 PARTITION OF fact_ventes
FOR VALUES FROM ('2024-01-01') TO ('2024-02-01');

CREATE TABLE fact_ventes_y2024m02 PARTITION OF fact_ventes
FOR VALUES FROM ('2024-02-01') TO ('2024-03-01');
```

## Partie 6 : Bonnes Pratiques et Pièges à Éviter

### ✅ Bonnes pratiques de modélisation :

1. **Grain cohérent** : Toutes les lignes de la table de faits doivent avoir le même niveau de détail
2. **Clés de substitution** : Toujours utiliser des SK pour l'indépendance et la performance
3. **Contraintes CHECK** : Valider les règles métier au niveau de la base de données
4. **Attributs dégénérés** : Stocker directement dans les faits les identifiants qui n'ont pas de dimension dédiée
5. **Dimensions conformes** : Réutiliser les mêmes dimensions entre différents schémas pour l'analyse cross-fonctionnelle

### ❌ Pièges courants à éviter :

1. **Mélanger les grains** : Ne pas mélanger tickets complets et lignes de produits dans la même table
2. **Mesures non additives** : Ne jamais stocker des ratios ou pourcentages directement dans les faits
3. **SCD mal gérées** : Oublier de gérer les changements dans les dimensions (prix, catégories, etc.)
4. **Attributs manquants** : Ne pas inclure les attributs nécessaires à l'analyse (segment client, type magasin, etc.)
5. **Performance négligée** : Oublier les index sur les clés étrangères et les filtres fréquents

### 🎯 Questions de qualité pour l'auto-évaluation :

1. **Qualité du grain** : Que se passe-t-il si une promotion s'applique au ticket entier plutôt qu'à la ligne produit ?
2. **Qualité des SCD** : Comment gérer un changement de prix de produit avec effet rétroactif ?
3. **Qualité des mesures** : Pourquoi le taux de remise ne doit-il pas être stocké dans les faits ?
4. **Qualité des dimensions** : Quand utiliser une hiérarchie (flocon) vs une dimension plate (étoile) ?
5. **Qualité de performance** : Quel impact sur les performances si on n'indexe pas les clés étrangères ?

## Partie 7 : Extensions Possibles et Prochaines Étapes

### 🚀 Extensions pour aller plus loin :

1. **Multi-grains** : Ajouter une table de faits agrégée au niveau ticket pour les analyses panier moyen
2. **Schéma en flocon** : Ajouter des sous-catégories dans `DIM_PRODUIT` ou des sous-régions dans `DIM_MAGASIN`
3. **Faits sans faits** : Créer une table de faits pour le suivi des stocks quotidiens
4. **Bridge tables** : Gérer les relations plusieurs-à-plusieurs (ex: produits ↔ promotions multiples)
5. **Time intelligence** : Ajouter des calculs de comparaisons (YoY, MoM, rolling averages)

### 📈 Prochaines étapes dans le parcours BI :

- **TD2** : Opérations OLAP et fonctions fenêtres pour l'analyse avancée
- **TD3** : Choix d'architecture (ROLAP vs MOLAP vs HOLAP) et benchmarking
- **Mini-projet** : Mise en œuvre complète d'un data warehouse retail

### 🎯 Compétences acquises avec ce TD1 avancé :

✅ **Conception** : Schéma en étoile complexe avec 6 dimensions  
✅ **Implémentation** : DDL complet avec contraintes et validation  
✅ **Qualité** : Gestion des SCD Type 2 et des attributs dégénérés  
✅ **Performance** : Indexation et partitionnement stratégique  
✅ **Analyse** : Requêtes multi-dimensions avec agrégations avancées  

---

**TD1 terminé** - Vous maîtrisez maintenant les fondamentaux avancés de la modélisation dimensionnelle !